In [6]:
import sys
sys.path.append('/home/shiftpub/Dynamic_AMM/exp')
from metrics_analysis import MetricsAnalysis
from metrics_plot import plot_value_and_vega_comparison, subplot_comparison
import numpy as np

output_dir = '/home/shiftpub/Dynamic_AMM/output/combined_metrics.parquet'


## 1. Expected Fee

In [ ]:
metric = 'expected_fee'
subplot_comparison(output_dir,
                metric=metric,
                drift=False,
                plot_rows=3,
                max_plots=1,
                value_type='value',
                subplot_by='sigma',
                max_sigma=2.0)
plot_value_and_vega_comparison(path=output_dir, metric=metric)

## 2. Pool Value

In [ ]:
metric = 'pool_value'
subplot_comparison(output_dir,
                metric=metric,
                drift=False,
                plot_rows=3,
                max_plots=1,
                value_type='value',
                subplot_by='sigma',
                max_sigma=2.0)
plot_value_and_vega_comparison(path=output_dir, metric=metric)

## 3. Trader PNL

In [ ]:
metric = 'trader_pnl'
subplot_comparison(output_dir,
                metric=metric,
                drift=False,
                plot_rows=3,
                max_plots=1,
                value_type='value',
                subplot_by='fee_rate')
plot_value_and_vega_comparison(path=output_dir, metric=metric)

## 4. Impermanent Loss

In [ ]:
metric = 'impermanent_loss'
subplot_comparison(output_dir,
                metric=metric,
                drift=False,
                plot_rows=3,
                max_plots=1,
                value_type='value',
                subplot_by='sigma',
                max_sigma=2.0)
plot_value_and_vega_comparison(path=output_dir, metric=metric)

In [6]:
## 5. Net Profit

In [ ]:
metric = 'net_profit'
subplot_comparison(output_dir,
                metric=metric,
                drift=False,
                plot_rows=3,
                max_plots=1,
                value_type='value',
                subplot_by='sigma',
                max_sigma=2.0)
plot_value_and_vega_comparison(path=output_dir, metric=metric)

## 6. Accounting Profit

In [ ]:
metric = 'accounting_profit'
subplot_comparison(output_dir,
                metric=metric,
                drift=False,
                plot_rows=3,
                max_plots=1,
                value_type='value',
                subplot_by='sigma',
                max_sigma=2.0)
plot_value_and_vega_comparison(path=output_dir, metric=metric)

In [ ]:
import numpy as np
from scipy.stats import lognorm
from scipy.integrate import quad
import matplotlib.pyplot as plt

def pnl_two_integrals_zero_drift(f: float, sigma: float, ls: float = 1.0) -> float:
    """Calculate E[PnL] using two separate integrals for zero drift case"""
    
    def integrand1(v, f, ls):
        # First integral: (1 - sqrt(v/(1-f)))^2
        return ls * (1 - np.sqrt(v/(1-f)))**2 * lognorm.pdf(v, s=sigma, scale=np.exp(-0.5*sigma**2))
    
    def integrand2(v, f, ls):
        # Second integral: (sqrt(v) - sqrt(1/(1-f)))^2
        return ls * (np.sqrt(v) - np.sqrt(1/(1-f)))**2 * lognorm.pdf(v, s=sigma, scale=np.exp(-0.5*sigma**2))
    
    result1, _ = quad(integrand1, 1e-10, 1-f, args=(f, ls))
    result2, _ = quad(integrand2, 1/(1-f), np.inf, args=(f, ls))
    return result1 + result2

def pnl_single_integral_zero_drift(f: float, sigma: float, ls: float = 1.0) -> float:
    """Calculate E[PnL] using the combined single integral for zero drift case"""
    
    def integrand(v, f, ls):
        # Combined integral: 2(1 - sqrt(v/(1-f)))^2
        return 2 * ls * (1 - np.sqrt(v/(1-f)))**2 * lognorm.pdf(v, s=sigma, scale=np.exp(-0.5*sigma**2))
    
    result, _ = quad(integrand, 1e-10, 1-f, args=(f, ls))
    return result

# Test parameters
test_params = [
    {'f': 0.001, 'sigma': 0.1},
    {'f': 0.003, 'sigma': 0.5},
    {'f': 0.005, 'sigma': 1.0},
    {'f': 0.01, 'sigma': 2.0}
]

# Compare results
print("Comparing Zero-Drift PnL calculations:")
print("\nf\t\tsigma\t\tTwo Integrals\tSingle Integral\tDifference")
print("-" * 70)

for params in test_params:
    result1 = pnl_two_integrals_zero_drift(params['f'], params['sigma'])
    result2 = pnl_single_integral_zero_drift(params['f'], params['sigma'])
    abs_diff = abs(result1 - result2)
    rel_error = abs_diff/abs(result1) if abs(result1) > 1e-10 else abs_diff
    
    print(f"{params['f']:.3f}\t\t{params['sigma']:.1f}\t\t{result1:.6f}\t{result2:.6f}\t{abs_diff:.6f}")

# Visual comparison
f_values = np.linspace(0.001, 0.01, 20)
sigma = 2.0

results1 = [pnl_two_integrals_zero_drift(f, sigma) for f in f_values]
results2 = [pnl_single_integral_zero_drift(f, sigma) for f in f_values]

plt.figure(figsize=(10, 6))
plt.plot(f_values, results1, 'b-', label='Two Integrals')
plt.plot(f_values, results2, 'r--', label='Single Integral')
plt.xlabel('Fee Rate (f)')
plt.ylabel('Expected PnL (Zero Drift)')
plt.title(f'Zero-Drift PnL Calculation Comparison (σ = {sigma})')
plt.legend()
plt.grid(True)
plt.show()

# Calculate relative errors
relative_errors = [abs(r1 - r2)/abs(r1) for r1, r2 in zip(results1, results2)]
print("\nMaximum relative error:", max(relative_errors))
print("Average relative error:", sum(relative_errors)/len(relative_errors))

In [ ]:
import numpy as np
from scipy.stats import lognorm
from scipy.integrate import quad
import matplotlib.pyplot as plt

def calculate_pnl_components(f: float, sigma: float, ls: float = 1.0):
    """Calculate both integrals separately and their sum"""
    
    def integrand1(v, f, ls):
        # First integral: (1 - sqrt(v/(1-f)))^2
        return ls * (1 - np.sqrt(v/(1-f)))**2 * lognorm.pdf(v, s=sigma, scale=np.exp(-0.5*sigma**2))
    
    def integrand2(v, f, ls):
        # Second integral: (sqrt(v) - sqrt(1/(1-f)))^2
        return ls * (np.sqrt(v) - np.sqrt(1/(1-f)))**2 * lognorm.pdf(v, s=sigma, scale=np.exp(-0.5*sigma**2))
    
    # Calculate first integral from 0 to 1-f
    result1, _ = quad(integrand1, 1e-4, 1-f, args=(f, ls))
    
    # Calculate second integral from 1/(1-f) to infinity
    result2, _ = quad(integrand2, 1/(1-f), np.inf, args=(f, ls))
    
    return result1, result2, result1 + result2

# Test parameters
f = 0.005  # Fixed fee rate
sigma_values = np.linspace(0.1, 6.0, 100)  # Range of sigma values

# Calculate results for different sigma values
results = [calculate_pnl_components(f, sigma) for sigma in sigma_values]
integral1_values = [r[0] for r in results]
integral2_values = [r[1] for r in results]
total_values = [r[2] for r in results]

# Create the visualization
plt.figure(figsize=(12, 8))

# Plot each component
plt.plot(sigma_values, integral1_values, 'b-', label='First Integral (0 to 1-f)', linewidth=2)
plt.plot(sigma_values, integral2_values, 'r-', label='Second Integral (1/(1-f) to ∞)', linewidth=2)
plt.plot(sigma_values, total_values, 'g--', label='Total PnL', linewidth=2)

plt.xlabel('Volatility (σ)')
plt.ylabel('Expected PnL')
plt.title(f'Components of Expected PnL vs Volatility (f = {f})')
plt.legend()
plt.grid(True)

# Add a table with numerical values
print("\nNumerical Values:")
print("σ\t\tFirst Integral\tSecond Integral\tTotal PnL")
print("-" * 60)
for i in range(len(sigma_values)):
    print(f"{sigma_values[i]:.2f}\t\t{integral1_values[i]:.6f}\t{integral2_values[i]:.6f}\t{total_values[i]:.6f}")

plt.show()


In [ ]:
import numpy as np
from scipy.stats import lognorm, norm
import matplotlib.pyplot as plt

def plot_lognormal_and_normal():
    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Range of x values
    x_lognormal = np.linspace(0.0001, 2, 1000)  # for lognormal
    x_normal = np.linspace(-2, 2, 1000)        # for normal
    
    # Different sigma values to show
    sigma_values = [0.1, 0.25, 0.5, 0.75]
    colors = ['blue', 'red', 'green', 'purple']
    
    # First subplot: Log-normal distribution
    for sigma, color in zip(sigma_values, colors):
        # For lognormal with μ = -σ²/2
        scale = np.exp(-0.5 * sigma**2)
        pdf_lognormal = lognorm.pdf(x_lognormal, s=sigma, scale=scale)
        
        ax1.plot(x_lognormal, pdf_lognormal, color=color, 
                label=f'σ = {sigma}', linewidth=2)
    
    # ax1.axvline(x=1, color='black', linestyle='--', alpha=0.3)
    ax1.set_xlabel('v')
    ax1.set_ylabel('Probability Density')
    ax1.set_title('Log-normal Distribution\n(μ = -σ²/2)')
    ax1.legend()
    ax1.grid(True)
    
    # Second subplot: Normal distribution (log transform)
    for sigma, color in zip(sigma_values, colors):
        # Normal distribution with μ = -σ²/2
        pdf_normal = norm.pdf(x_normal, loc=-0.5*sigma**2, scale=sigma)
        
        ax2.plot(x_normal, pdf_normal, color=color, 
                label=f'σ = {sigma}, μ = {-0.5*sigma**2:.2f}', linewidth=2)
    
    # ax2.axvline(x=0, color='black', linestyle='--', alpha=0.3)
    ax2.set_xlabel('ln(v)')
    ax2.set_ylabel('Probability Density')
    ax2.set_title('Normal Distribution\n(after taking natural log)')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()

    # Print some key statistics
    print("\nKey Statistics:")
    print("σ\tLognormal\t\t\tNormal")
    print("\tMean\tMedian\tMode\t\tMean\tMedian/Mode")
    print("-" * 70)
    for sigma in sigma_values:
        # Lognormal stats
        ln_mean = 1.0  # always 1 due to μ = -σ²/2
        ln_median = np.exp(-0.5 * sigma**2)
        ln_mode = np.exp(-0.5 * sigma**2 - sigma**2)
        
        # Normal stats (after log transform)
        n_mean = -0.5 * sigma**2  # μ
        n_median = n_mean  # median = mode = mean for normal distribution
        
        print(f"{sigma:.1f}\t{ln_mean:.3f}\t{ln_median:.3f}\t{ln_mode:.3f}\t\t{n_mean:.3f}\t{n_median:.3f}")

# Create the plots
plot_lognormal_and_normal()